# Google Spanner
> [Spanner](https://cloud.google.com/spanner) 是一个可高度扩展的数据库，它将无限的可扩展性与关系语义结合起来，例如二级索引、强一致性、模式和 SQL，在一个简单的解决方案中提供 99.999% 的可用性。

本笔记将介绍如何使用 `Spanner` 和 `SpannerVectorStore` 类进行向量搜索。

在 [GitHub](https://github.com/googleapis/langchain-google-spanner-python/) 上了解更多关于该包的信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/docs/vector_store.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Cloud Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)
* [创建 Spanner 实例](https://cloud.google.com/spanner/docs/create-manage-instances)
* [创建 Spanner 数据库](https://cloud.google.com/spanner/docs/create-manage-databases)

### 🦜🔗 库安装
集成内容位于其自身的 `langchain-google-spanner` 包中，因此需要安装它。

In [1]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


**仅限 Colab：** 取消注释以下单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [2]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 身份验证
使用当前登录到此笔记本的 IAM 用户身份验证到 Google Cloud，以便访问您的 Google Cloud 项目。

* 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续操作。
* 如果您正在使用 Vertex AI Workbench，请查看此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便您可以在此笔记本中使用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请参阅支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

### 💡 API 启用
`langchain-google-spanner` 包要求您在 Google Cloud 项目中[启用 Spanner API](https://console.cloud.google.com/flows/enableapi?apiid=spanner.googleapis.com)。

In [ ]:
# enable Spanner API
!gcloud services enable spanner.googleapis.com

## 基本用法

### 设置 Spanner 数据库值
在 [Spanner 实例页面](https://console.cloud.google.com/spanner?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687)查找您的数据库值。

In [4]:
# @title Set Your Values Here { display-mode: "form" }
INSTANCE = "my-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "vectors_search_data"  # @param {type: "string"}

### 初始化表
`SpannerVectorStore` 类实例需要一个包含 id、content 和 embeddings 列的数据库表。

可以使用 `init_vector_store_table()` 辅助方法为您创建具有正确架构的表。

In [ ]:
from langchain_google_spanner import SecondaryIndex, SpannerVectorStore, TableColumn

SpannerVectorStore.init_vector_store_table(
    instance_id=INSTANCE,
    database_id=DATABASE,
    table_name=TABLE_NAME,
    # Customize the table creation
    # id_column="row_id",
    # content_column="content_column",
    # metadata_columns=[
    #     TableColumn(name="metadata", type="JSON", is_null=True),
    #     TableColumn(name="title", type="STRING(MAX)", is_null=False),
    # ],
    # secondary_indexes=[
    #     SecondaryIndex(index_name="row_id_and_title", columns=["row_id", "title"])
    # ],
)

### 创建 embedding 实例

您可以使用任何 [LangChain embedding 模型](/docs/integrations/text_embedding/)。
您可能需要启用 Vertex AI API 才能使用 `VertexAIEmbeddings`。我们建议为生产环境设置 embedding 模型的版本，了解更多关于 [文本 embedding 模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings) 的信息。

In [ ]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [5]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### SpannerVectorStore

初始化 `SpannerVectorStore` 类需要提供 4 个必需参数，其他参数为可选参数，仅在需要与默认值不同时才需传入。

1.  `instance_id` - Spanner 实例的名称。
1.  `database_id` - Spanner 数据库的名称。
1.  `table_name` - 数据库中用于存储文档及其嵌入（embeddings）的表的名称。
1.  `embedding_service` - 用于生成嵌入（embeddings）的 Embeddings 实现。

In [ ]:
db = SpannerVectorStore(
    instance_id=INSTANCE,
    database_id=DATABASE,
    table_name=TABLE_NAME,
    embedding_service=embeddings,
    # Connect to a custom vector store table
    # id_column="row_id",
    # content_column="content",
    # metadata_columns=["metadata", "title"],
)

#### 添加文档
在向量存储中添加文档。

In [ ]:
import uuid

from langchain_community.document_loaders import HNLoader

loader = HNLoader("https://news.ycombinator.com/item?id=34817881")

documents = loader.load()
ids = [str(uuid.uuid4()) for _ in range(len(documents))]
db.add_documents(documents, ids)

#### 搜索文档
使用相似性搜索在向量存储中搜索文档。

In [ ]:
db.similarity_search(query="Explain me vector store?", k=3)

#### 搜索文档
通过最大边际相关性搜索在向量存储中搜索文档。

In [ ]:
db.max_marginal_relevance_search("Testing the langchain integration with spanner", k=3)

#### 删除文档
要从向量存储中删除文档，请使用初始化 VectorStore 时对应 `row_id` 列中的值的 ID。

In [ ]:
db.delete(ids=["id1", "id2"])

#### 删除文档
要从向量存储中移除文档，可以使用这些文档本身。在初始化 VectorStore 时提供的 content 列和 metadata 列将用于查找与文档对应的行。任何匹配的行都将被删除。

In [ ]:
db.delete(documents=[documents[0], documents[1]])